# Sport-73 Classification

Let's do the imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing import image
from tensorflow.keras import (Input, Model, layers, losses, optimizers, metrics, utils, models)

sns.set_style('darkgrid')

Fixing some constants that may need multiple times.

In [ ]:
# CONSTANTS
DATA_PATH = '../input/sports-classification'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
VAL_PATH = os.path.join(DATA_PATH, 'valid')
TEST_PATH = os.path.join(DATA_PATH, 'test')

IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = (224, 224, 3)
NUM_CLASSES = len(os.listdir(TEST_PATH))

# HYPERPARAMETERS
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-3

Creating the image data generators.

In [ ]:
data_generator = image.ImageDataGenerator(rescale = 1./255)
train_generator = data_generator.flow_from_directory(directory= TRAIN_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)
val_generator = data_generator.flow_from_directory(directory= VAL_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)
test_generator = data_generator.flow_from_directory(directory= TEST_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE,
                                                    shuffle= False)

First, we are going to try with a simple CNN architecture and see how it does

In [ ]:
def get_cnn_model(IMAGE_SHAPE):
    """
    creates and returns a CNN model.
    """
    # Define the tensors for the two input images
    input_layer = Input(IMAGE_SHAPE)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu")(input_layer)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(NUM_CLASSES, name="final_dense", activation='softmax')(x)
    return Model(input_layer, outputs)

In [ ]:
model = get_cnn_model(IMAGE_SHAPE)
model.summary()

In [ ]:
model.compile(optimizer = optimizers.Adam(LEARNING_RATE), 
                loss = losses.categorical_crossentropy, 
                metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data= val_generator, epochs = EPOCHS)

In [ ]:
model.save('model1')

In [ ]:
model.evaluate(test_generator)

As we can see, the results look horrible and whatever we are doing is not working. 
We should now try with some transfer learning and see if previous learning helps with the results.

- We will be using MobileNet architecture with some added modifications.
- Image augmentation can also be useful for the model, especially one with 73 classes.

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
train_data_generator = image.ImageDataGenerator(
                                            rotation_range=30,
                                            zoom_range=0.15,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.15,
                                            horizontal_flip=True,
                                            fill_mode="nearest",
                                            preprocessing_function=preprocess_input
                                            )

test_data_generator = image.ImageDataGenerator(
                                            preprocessing_function=preprocess_input
                                            )

train_generator = train_data_generator.flow_from_directory(directory= TRAIN_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)
val_generator = train_data_generator.flow_from_directory(directory= VAL_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)
test_generator = test_data_generator.flow_from_directory(directory= TEST_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'categorical',
                                                    batch_size= BATCH_SIZE)

Let's just add two Dense layers to the architecture.

In [ ]:
class ModifiedMobileNet():
    '''
    This class creates the mobilenet model.
    '''
    def __init__(self, input_shape, nb_classes):
        self.input_shape = input_shape
        self.nb_classes = nb_classes
        
    def get_model(self, unfreeze_layers = None, lr_rate = 0.001):
        # load base model
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=self.input_shape)
        
        # freezing the layers
        for layer in (base_model.layers) if not unfreeze_layers else (base_model.layers[:-int(unfreeze_layers)]):
            layer.trainable = False
            
        inputs = Input(shape=self.input_shape)
        x = base_model(inputs, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        outputs = layers.Dense(self.nb_classes, activation='softmax')(x)
        model = Model(inputs, outputs)
        
        # model compilation
        optimizer = optimizers.Adam(learning_rate=lr_rate)
        model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['accuracy'])
        return model

 We'll keep the base MobileNet frozen completely and only the added layers will learn from our dataset.

In [ ]:
mobilenet = ModifiedMobileNet(input_shape= IMAGE_SHAPE, nb_classes= NUM_CLASSES)
model = mobilenet.get_model(unfreeze_layers=0, lr_rate= LEARNING_RATE)

In [ ]:
model.compile(optimizer = optimizers.Adam(LEARNING_RATE), 
                loss = losses.categorical_crossentropy, 
                metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data= val_generator, epochs = EPOCHS)

In [ ]:
model.save('model2')

In [ ]:
model.evaluate(test_generator)

Let's check the Precision and Recall of Both architectures.  
Since there are 73 classes, we will calculated weighted Precision and Recall for ease of understanding.

In [ ]:
test_generator = data_generator.flow_from_directory(directory= TEST_PATH,
                                                    target_size=IMAGE_SIZE,
                                                    color_mode= 'rgb',
                                                    class_mode= 'sparse',
                                                    batch_size= 365,
                                                    shuffle= False)

In [ ]:
test_images, test_labels = test_generator.next()

In [ ]:
default_model = models.load_model('model1/')
improved_model = models.load_model('model2/')

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
def_precision, def_recall = (precision_score(test_labels, default_predictions, average= 'weighted'),
                            recall_score(test_labels, default_predictions, average= 'weighted'))
imp_precision, imp_recall = (precision_score(test_labels, improved_predictions, average= 'weighted'),
                            recall_score(test_labels, improved_predictions, average= 'weighted'))